In [3]:
import scipy
scipy.__version__

'1.1.0'

In [13]:
from MDAnalysis.lib.pkdtree import PeriodicKDTree
import numpy as np
from MDAnalysis.lib.distances import self_distance_array,distance_array
import scipy
from scipy.spatial.distance import squareform
import itertools

##Initialize the class for benchmarks
class SimpleKDBench():
    """
    Class for simple KDtree benchmarks
    """
    def __init__(self):
        #self.box = np.array([100.,100.,100.,90.,90.,90.],dtype=np.float32)
        self.box = np.ones(3).astype(np.float32)*100
        self.points = np.random.uniform(low=0,high=1.0,size=(10000,3))*self.box[:3]
        self.points = np.array(self.points,dtype=np.float32)
        self.maxdist = 10.0
    
    def kdtree_distance(self,points,box,maxdist):
        kdtree = PeriodicKDTree(box,bucket_size=10)
        kdtree.set_coords(points)
        pair,distances = list(),list()
        for idx,centers in enumerate(points):
            kdtree.search(centers,maxdist)
            indices = kdtree.get_indices()
            indices = [i for i in indices if i > idx]
            distances += list(distance_array(centers.reshape((1,3)),points[indices],box).flatten())
            pair += list(itertools.product(([idx]),indices))    
        return tuple((np.asarray(pair),np.asarray(distances)))
    
    def bf_select(self, points, box, maxdist):
        distance = self_distance_array(points,box)
        distance = scipy.spatial.distance.squareform(distance)
        distance[np.tril_indices(distance.shape[0])] = (1.1*maxdist)
        mask = np.where((distance < maxdist))
        out = tuple((np.column_stack((mask[0],mask[1])),distance[mask]))
        return out

In [14]:
ob = SimpleKDBench()
kd = ob.kdtree_distance(ob.points, ob.box, ob.maxdist)
print(kd)

(array([[   0,  160],
       [   0,  630],
       [   0,  880],
       ...,
       [9952, 9980],
       [9969, 9973],
       [9986, 9991]]), array([7.1049704 , 6.95386775, 9.70376903, ..., 4.61587456, 9.72549188,
       9.93232761]))


In [16]:
p = ob.bf_select(ob.points,ob.box,ob.maxdist)
p

(array([[   0,  160],
        [   0,  630],
        [   0,  880],
        ...,
        [9952, 9980],
        [9969, 9973],
        [9986, 9991]]),
 array([7.1049704 , 6.95386775, 9.70376903, ..., 4.61587456, 9.72549188,
        9.93232761]))

In [17]:
#Testing
np.testing.assert_almost_equal(p[1],kd[1]) #Distances
np.testing.assert_almost_equal(p[0],kd[0]) #Selections

In [18]:
%timeit ob.kdtree_distance(ob.points, ob.box, ob.maxdist)

1.97 s ± 38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit ob.bf_select(ob.points,ob.box,ob.maxdist)

5.14 s ± 87.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
